# Quickstart

## Installation

CredoAI Lens can be installed with pip from PyPI as follows:

`pip install credoai`

Optional dependencies can be installed as follows:

`pip install credoai[extras]`

## Overview

CredoAI Lens is the assessment framework component of the broader CredoAI suite.
It is usable as a standalone gateway to assessments or in combination
with CredoAI's Governance Platform. 

Understanding how your AI systems are operating is the most important step in intervening upon your system. From the technically complicated questions of improving a system to the business-relevant questions of whether to deploy a system - everything is fundamentally founded upon good observability. Lens strives to make assessment comprehensive, easy, and adaptable.

### Assessments

CredoAI Lens is an entry point to assessments developed by CredoAI, as well as the broader ecosystem of open-source assessments. Custom analytics can also be folded in easily (see the `lens customization` notebook)

AI system assessment starts with verifying standard performance metrics to an evolving set of assessments falling under the banner of *Responsible AI*. A non-exhaustive list includes

* Fairness
* Explainability
* Performance
* Robustness

These different categories of assessment differ substantially based on whether one is 
evaluating datasets or models, what kind of model (e.g., tabular, NLP, computer vision), and the use-case. As the ecosystem develops, Lens will support assessing a broader range of AI systems. Currently, we are focused on Fairness.

### Governance

While Lens is a stand-alone assessment framework, its value is increased when combined with the CredoAI Governance Platform. The platform supports multi-stakeholder `Alignment` on how to assess your AI systems (e.g., what does good look like for this system?). It also supports translating assessment results into a Risk perspective that is scalable across your organization and understandable to diverse stakeholders.



## Lens in 5 minutes

Get your data and models together! Here we have a support vector machine trained on the Iris Dataset. No train/test split needed for this tutorial!

In [1]:
# imports for example data and model training
from credoai.data import fetch_creditdefault
from sklearn.ensemble import GradientBoostingClassifier

# Base Lens imports
from credoai.lens import Lens, CredoModel, CredoData
# importing a particular assessment
from credoai.assessment.assessments import FairnessAssessment



Failed to import tensorflow. Please note that tensorflow is not installed by default when you install tensorflow_hub. This is so that users can decide which tensorflow package to use. To use tensorflow_hub, please install a current version of tensorflow by following the instructions at https://tensorflow.org/install and https://tensorflow.org/hub/installation.




In [2]:
data = fetch_creditdefault(as_frame=True)
X = data['data'].drop(columns=['SEX'])
y = data['target']
sensitive_feature = data['data']['SEX']

In [3]:
model = GradientBoostingClassifier()
model.fit(X,y)

GradientBoostingClassifier()

### Using Lens

Below is a basic example where our goal is to evaluate the fairness of the above model. We will rely on Lens defaults for this analysis.

#### Preparing artifacts

Lens interacts with Credo Artifacts which wrap models and datasets and standardizes them for use by different assessments.
Below we create a `CredoModel` object, which automatically infers that the "model" object is from scikit-learn. We also create a `CredoData` object which is store X, y and sensitive features.

In [4]:
credo_model = CredoModel(name='credit_default_classifier',
                         model=model)

# no sensitive feature! need to correct. Better dataset needed
credo_data = CredoData(name='UCI-credit-default',
                       X=X, 
                       y=y.astype(int),
                       sensitive_features=sensitive_feature)

#### Alignment Manifest

The Alignment Manifest describes how the assessments should be run at a high level. It should be thought of as the output of a multi-stakeholder alignment session on "how we should assess this AI system". Another way to think about it is as the *paramterization* of the assessments Lens will run.

The Alignment Manifest is a *subset* of all possible parameterizations of the different assessments you will run. You have more programmatic control of the assessments by making use of the `init_assessment_kwargs` (passed to Lens when initializied) or the `assessment_kwargs`, which are passed to the `run_assessments` function.

If you use the Credo AI Governance Platform, the alignment manifest is a principle artifact determined during the *Alignment Phase*. It is the output of multi-stakeholder collaboration. Lens will automatically download the Alignment Manifest associated with your governance credentials (which uses another artifact: `CredoGovernance`)

In [5]:
# specify the metrics that will be used by the FairnessBase assessment
alignment_manifest = {'metrics': ['precision_score']}

#### Run Lens

Once we have the model and data artifacts, as well as the manifest, we can run Lens

In [6]:
lens = Lens(model=credo_model,
            data=credo_data,
            assessments=[FairnessAssessment()],
            manifest=alignment_manifest)

Lens has one primary method: `run_assessments`

`run_assessments` outputs the results into a dictionary that can be used for further processing. If the `export` keyword is used, it can also export the data to a json or straight to Credo AI's governance platform (again, if you are using it!)

In [7]:
results = lens.run_assessments()

In [8]:
results['FairnessBase']['fairness']

,value
precision_score,0.000543


Example export to `~/credoai_test_location`. Uncomment to test

In [10]:
# # example export
# import os
# output_directory = os.path.join(os.path.expanduser('~'), 'credoai_test_location')
# lens.run_assessments(export=output_directory)

{'FairnessBase': {'fairness':                     value
  precision_score  0.000543,
  'disaggregated_results':          precision_score
  SEX                     
  1.0             0.695954
  2.0             0.695411
  overall         0.695652}}